In [ ]:
!pip install optuna

In [ ]:
!pip install lightgbm

In [3]:
import shap
import optuna
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
import gc
from sklearn.metrics import roc_auc_score, f1_score
from sklearn.linear_model import LogisticRegression, SGDClassifier
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation, reset_parameter
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold
import numpy as np
from time import time
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from xgboost import XGBClassifier

In [4]:
df_train = pd.read_parquet("/kaggle/input/data-tour/train.parquet")
df_train

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag,flag
0,1678548,3,8,7,17,16,9,1,9,2,...,0,0,1,1,3,2,1,0,0,0
1,2834188,11,3,2,0,7,14,8,2,5,...,0,0,1,1,2,4,1,0,0,0
2,811902,11,9,6,11,13,14,8,2,5,...,3,3,4,1,2,4,1,0,0,0
3,836450,1,16,0,13,0,4,9,5,2,...,3,3,4,1,3,4,1,0,0,0
4,1769024,15,9,9,4,8,1,11,1,2,...,3,3,4,1,2,4,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17659828,2103608,4,1,9,4,8,1,11,10,2,...,3,3,4,1,2,3,1,1,1,0
17659829,2169551,7,15,4,12,10,6,13,15,2,...,0,0,1,1,3,2,1,0,0,0
17659830,1613436,1,8,9,16,5,9,9,10,3,...,3,3,4,1,3,4,1,0,0,0
17659831,1604231,6,17,13,12,1,2,4,2,2,...,0,0,1,1,3,0,1,0,0,0


In [5]:
df_test = pd.read_parquet("/kaggle/input/data-tour/test.parquet")
df_test

,id,rn,pre_since_opened,pre_since_confirmed,pre_pterm,pre_fterm,pre_till_pclose,pre_till_fclose,pre_loans_credit_limit,pre_loans_next_pay_summ,...,enc_paym_22,enc_paym_23,enc_paym_24,enc_loans_account_holder_type,enc_loans_credit_status,enc_loans_credit_type,enc_loans_account_cur,pclose_flag,fclose_flag,id_x_rn
0,1472943,2,10,4,16,16,13,10,10,2,...,3,3,4,1,3,4,1,0,0,1472943_x_2
1,660465,5,6,14,9,7,3,11,0,2,...,3,3,4,1,3,4,1,0,0,660465_x_5
2,1788193,3,1,0,9,9,7,2,16,2,...,3,3,4,1,3,4,1,0,0,1788193_x_3
3,2767146,3,15,8,1,16,6,13,14,2,...,3,3,4,1,3,3,1,0,0,2767146_x_3
4,2601698,9,19,10,4,8,1,11,4,2,...,3,3,4,1,2,3,1,1,1,2601698_x_9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
654063,2159273,7,9,1,15,9,1,11,6,2,...,3,3,4,1,3,4,1,0,0,2159273_x_7
654064,320081,7,0,4,3,5,6,13,13,2,...,3,3,4,1,3,4,1,0,0,320081_x_7
654065,2663677,4,15,4,1,15,13,13,7,5,...,3,3,4,1,3,4,1,0,0,2663677_x_4
654066,1382804,18,11,16,11,13,14,8,14,0,...,0,3,4,1,2,4,1,0,0,1382804_x_18


In [5]:
set(df_train["id"]) == set(df_test["id"])

False

In [9]:
# Retirer la colonne 'flag' si nécessaire
df_train_check = df_train.drop(columns=["flag", "rn", "id"])
df_test_check = df_test.drop(columns=["id_x_rn", "rn", "id"])

# Vérifier ligne par ligne si la ligne de df_test est dans df_train
mask = df_test_check.apply(tuple, axis=1).isin(df_train_check.apply(tuple, axis=1))

print(mask)  # Série booléenne ligne par ligne
print(mask.sum(), "lignes de df_test sont déjà dans df_train")

0         False
1         False
2          True
3          True
4          True
          ...  
654063     True
654064     True
654065    False
654066    False
654067     True
Length: 654068, dtype: bool
343437 lignes de df_test sont déjà dans df_train


In [7]:
mask = df_test.isin(df_train.drop(columns = ["flag"]))
print(mask)  # Série booléenne
print(mask.sum(), "lignes de df_test sont déjà dans df_train")


           id     rn  pre_since_opened  pre_since_confirmed  pre_pterm  \
0       False  False             False                False      False   
1       False  False             False                False      False   
2       False  False             False                False      False   
3       False  False             False                False      False   
4       False  False             False                False       True   
...       ...    ...               ...                  ...        ...   
654063  False  False             False                False      False   
654064  False   True              True                False      False   
654065  False  False             False                False      False   
654066  False  False             False                False      False   
654067  False  False             False                False      False   

        pre_fterm  pre_till_pclose  pre_till_fclose  pre_loans_credit_limit  \
0            True            Fal

In [ ]:
df_train["flag"].value_counts()

In [ ]:
# Create a mapping of old_id → new unique integer
id_map = {old: new for new, old in enumerate(df_train['id'].unique(), start=1)}

# Apply the mapping
df_train['new_id'] = df_train['id'].map(id_map)

In [ ]:
df_train['id_count'] = df_train.groupby('id')['id'].transform('count')

In [ ]:
df_train.groupby("id_count")["flag"].count()

In [ ]:
df_train[df_train["id_count"] == 1].count()

In [ ]:
df_train["id"].max()

In [ ]:
paym_cols = [f'enc_paym_{i}' for i in range(25)]

# 1️⃣ Somme des retards
df_train['total_delays_25m'] = df_train[paym_cols].sum(axis=1)

# 2️⃣ Score pondéré (plus récent = plus important)
weights = np.arange(25, 0, -1)  # [25, 24, ..., 1]
df_train['weighted_delay_score'] = df_train[paym_cols].mul(weights, axis=1).sum(axis=1)

# Normaliser le score pondéré
df_train['weighted_delay_score_norm'] = df_train['weighted_delay_score'] / weights.sum()

df_train[["total_delays_25m", "weighted_delay_score", "weighted_delay_score_norm"]]

In [ ]:
bool_cols = ['is_zero_loans5', 'is_zero_loans530', 'is_zero_loans90']

# Pondérations
weights = [0.2, 0.3, 0.5]

# Calcul du score de risque
df_train['risk_score'] = (
    df_train['is_zero_loans5'] * weights[0] +
    df_train['is_zero_loans530'] * weights[1] +
    df_train['is_zero_loans90'] * weights[2]
)
df_train["risk_score"]

In [ ]:
# Score de risque combiné (pondéré)
df_train['credit_risk_score'] = (
    0.5 * df_train['pre_util'] +
    0.3 * df_train['pre_over2limit'] +
    0.2 * df_train['pre_maxover2limit']
)

# Catégorisation du risque (0=faible, 3=élevé)
bins = [0, 5, 10, 15, 21]
labels = [0, 1, 2, 3]
df_train['credit_risk_level'] = pd.cut(df_train['credit_risk_score'], bins=bins, labels=labels, include_lowest=True).astype(int)
df_train[["credit_risk_level", "credit_risk_score"]]

In [ ]:
df_train['cost_to_limit'] = df_train['pre_loans_credit_cost_rate'] / df_train['pre_loans_credit_limit']
df_train['pterm_times_limit'] = df_train['pre_pterm'] * df_train['pre_loans_credit_limit']
df_train['confirmation_to_open_days'] = df_train['pre_since_confirmed'] - df_train['pre_since_opened']

In [ ]:
df_train['util_to_limit'] = df_train['pre_util'] / (df_train['pre_loans_credit_limit'] + 1)
df_train['delays_to_term'] = df_train['total_delays_25m'] / (df_train['pre_pterm'] + 1)

In [ ]:
important_feat = ["id",'enc_paym_2', "rn", "enc_paym_0", "enc_paym_1", 'enc_loans_credit_type', 'pre_util','pre_loans_credit_cost_rate', 'is_zero_loans530', 'pre_loans_credit_limit',
 'pre_since_opened', 'is_zero_util', 'pre_pterm', 'is_zero_loans5', 'enc_paym_18', 'is_zero_loans90', 'pre_since_confirmed', "weighted_delay_score_norm",
 "total_delays_25m", "risk_score", "credit_risk_level", "credit_risk_score", "confirmation_to_open_days",
                 "pterm_times_limit", "cost_to_limit", "util_to_limit", "delays_to_term" ]

In [ ]:
df_train.drop(["enc_paym_17", "enc_paym_15","pre_loans5", "enc_paym_18", 
               "is_zero_maxover2limit", "is_zero_over2limit", "is_zero_loans6090",
               "enc_loans_account_cur", "pre_loans90", "pre_loans3060", 
               "pre_loans6090", "pre_loans_total_overdue", "enc_paym_22", "enc_paym_21"], axis = 1, inplace = True)

In [ ]:
df_train.drop(["is_zero_loans6090", "is_zero_over2limit", "enc_loans_account_cur", "pre_loans90", "pre_loans3060",
 "pre_loans6090", "pre_loans_total_overdue"], axis = 1, inplace = True)

In [ ]:
df_train["time_to_close_gap"] = df_train["pre_till_fclose"] - df_train["pre_till_pclose"]
df_train["loan_duration"] = df_train["pre_fterm"] - df_train["pre_pterm"]
df_train['confirmation_to_open_days'] = df_train['pre_since_confirmed'] - df_train['pre_since_opened']

In [ ]:
paym_cols = [f'enc_paym_{i}' for i in range(10)]

# 1️⃣ Somme des retards
df_train['total_delays_25m'] = df_train[paym_cols].sum(axis=1)

# 2️⃣ Score pondéré (plus récent = plus important)
weights = np.arange(10, 0, -1)  # [25, 24, ..., 1]
df_train['weighted_delay_score'] = df_train[paym_cols].mul(weights, axis=1).sum(axis=1)

In [ ]:
df_train["limit_x_util"] = df_train["pre_loans_credit_limit"] * df_train["pre_util"]
df_train["cost_x_util"] = df_train["pre_loans_credit_cost_rate"] * df_train["pre_util"]
df_train["gap_x_duration"] = df_train["time_to_close_gap"] * df_train["loan_duration"]

In [ ]:
df_train['limit_x_util'] = df_train['pre_loans_credit_limit'] * df_train['pre_util']
df_train['cost_x_util'] = df_train['pre_loans_credit_cost_rate'] * df_train['pre_util']

In [4]:
X_train, X_val, y_train, y_val = train_test_split(df_train.drop(["flag"],axis = 1), df_train.flag, stratify=df_train.flag, test_size=0.05, random_state = 42)
#del df_train
#_= gc.collect()

In [ ]:
X_val, X_val2, y_val, y_val2 = train_test_split(X_val, y_val, stratify=y_val, test_size=0.20)
#del df_train
#_= gc.collect()

In [ ]:
cluster_features = [
    "rn", 'pre_util', 'pre_over2limit', 'pre_maxover2limit',
    'pre_loans_credit_limit', 'pre_loans_credit_cost_rate',
    'pre_since_opened', 'pre_since_confirmed'
]

In [ ]:
cluster_features_payment = ['enc_paym_0','enc_paym_1', 'enc_paym_2','enc_paym_3','enc_paym_4',
 'enc_paym_5', 'enc_paym_6', 'enc_paym_7', 'enc_paym_8', 'enc_paym_9', "enc_paym_10", 
                            "enc_paym_11", "enc_paym_12"]
cluster_features_payment

In [ ]:
scaler2 = StandardScaler().set_output(transform = "pandas")
X_scaled2 = scaler2.fit_transform(X_train[cluster_features_payment])

In [ ]:
from sklearn.metrics import silhouette_score
from sklearn.cluster import KMeans

inertias2 = []
sil_scores = []
K_range = range(2, 14)  # de 2 à 13 clusters

for k in K_range:
    print(f"Calcul pour k={k}")
    kmeans2 = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans2.fit(X_scaled2)  # obtenir les labels pour silhouette
    inertias2.append(kmeans2.inertia_)
    


In [ ]:
kmeans2 = KMeans(n_clusters=13, random_state=42, n_init=10)
kmeans2.fit(X_scaled2)  

In [ ]:
X_val_scaled = scaler2.transform(X_val[cluster_features_payment])
X_val['cluster_payment_k13'] = kmeans2.predict(X_val_scaled)

In [ ]:
X_val2_scaled = scaler2.transform(X_val2[cluster_features_payment])
X_val2['cluster_payment_k13'] = kmeans2.predict(X_val2_scaled)

In [ ]:
X_train['cluster_payment_k13'] = kmeans2.predict(X_scaled2)

In [ ]:
scaler = StandardScaler().set_output(transform = "pandas")
X_scaled = scaler.fit_transform(X_train[cluster_features])

In [ ]:
X_scaled

In [ ]:
inertias = []
K_range = range(2, 14)  # entre 2 et 15 clusters (modifiable)

for k in K_range:
    print(k)
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)

In [ ]:
inertias

In [ ]:
K_range = [ x for x in range(2, 15)]
K_range

In [ ]:
# === 4️⃣ Visualiser le coude ===
plt.figure(figsize=(8,5))
plt.plot(K_range, inertias2, marker='o')
plt.title('Méthode du coude (Elbow Method)')
plt.xlabel('Nombre de clusters (k)')
plt.ylabel('Inertie (Within-Cluster Sum of Squares)')
plt.grid(True)
plt.show()

In [ ]:
# Entraînement final avec k = 13
kmeans = KMeans(n_clusters=13, random_state=42, n_init=10)
kmeans.fit(X_scaled)

In [ ]:
X_train['cluster_k13'] = kmeans.predict(X_scaled)

In [ ]:
X_train[cluster_features] = X_scaled

In [ ]:
X_train

In [ ]:
X_val_scaled = scaler.transform(X_val[cluster_features])
X_val['cluster_k13'] = kmeans.predict(X_val_scaled)

In [ ]:
X_val_scaled2 = scaler.transform(X_val2[cluster_features])
X_val2['cluster_k13'] = kmeans.predict(X_val_scaled2)

In [ ]:
X_val_scaled = scaler2.transform(X_val[cluster_features_payment])
X_val['cluster_payment_k13'] = kmeans2.predict(X_val_scaled)

In [ ]:
X_val2_scaled = scaler2.transform(X_val2[cluster_features_payment])
X_val2['cluster_payment_k13'] = kmeans2.predict(X_val2_scaled)

In [ ]:
X_train['cluster_payment_k13'] = kmeans2.predict(X_scaled2)

In [ ]:
X_train

In [ ]:
X_val

In [ ]:
# 1️⃣ Choisir les features à clusteriser
cluster_features = [
    'pre_util', 'pre_over2limit', 'pre_maxover2limit',
    'pre_loans_credit_limit', 'pre_loans_credit_cost_rate',
    'pre_since_opened', 'pre_since_confirmed'
]

# 2️⃣ Standardiser
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_train[cluster_features])

In [ ]:
# Create a new column with the count of duplicates
X_train['id_count'] = X_train.groupby('id')['id'].transform('count')
X_val['id_count'] = X_val.groupby('id')['id'].transform('count')

In [ ]:
# Creating a mapping of old_id → new unique integer
id_map_train = {old: new for new, old in enumerate(X_train['id'].unique(), start=1)}

# Applying the mapping
X_train['new_id'] = X_train['id'].map(id_map_train)

In [ ]:
# Create a mapping of old_id → new unique integer
id_map_val = {old: new for new, old in enumerate(X_val['id'].unique(), start=1)}

# Apply the mapping
X_val['new_id'] = X_val['id'].map(id_map_val)

In [ ]:
X_train[["id", "new_id"]]

In [ ]:
X_train.drop("id", axis = 1, inplace = True)
X_val.drop("id", axis = 1, inplace = True)

In [ ]:
X_train["Several_demand"] = X_train["id_count"].apply(lambda x : 0 if x < 2 else 1)

In [ ]:
X_val["Several_demand"] = X_val["id_count"].apply(lambda x : 0 if x < 2 else 1)

In [ ]:
# Garder seulement 20% des données d'entraînement
X_train_small, _, y_train_small, _ = train_test_split(
    X_train, y_train, test_size=0.9, random_state=42, stratify=y_train
)

# Garder seulement 20% des données de test
X_test_small, _, y_test_small, _ = train_test_split(
    X_val, y_val, test_size=0.9, random_state=42, stratify=y_val
)

print(f"Train subset size: {X_train_small.shape}, Test subset size: {X_test_small.shape}")

In [ ]:
# Base model with fixed hyperparameters
base_model = lgb.LGBMClassifier(
    boosting_type='goss',
    num_leaves=50,
    subsample=0.7571512794863664,
    colsample_bytree=0.6952547009268857,
    reg_alpha=0.002004697689738107,
    reg_lambda=6.691298345684395e-07,
    min_child_samples=60,
    class_weight=(1 / y_test_small.value_counts(normalize=True)).to_dict(),
    random_state=42
)

# Define the grid for tuning
param_grid = {
    "boosting_type" : ['goss', "gbt"],
    'n_estimators': [800, 1200, 1800],
    'learning_rate': [0.01, 0.05],
    'max_depth': [5, 7, 9]
}

# GridSearchCV setup
grid_search = GridSearchCV(
    estimator=base_model,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=3,
    verbose=2,
    n_jobs=1   # instead of -1
)


# Fit with validation monitoring
grid_search.fit(
    X_train_small, y_train_small,
    eval_set=[(X_test_small, y_test_small)],
    eval_metric='auc',
    callbacks=[early_stopping(stopping_rounds=50), log_evaluation(100)]
)

# Results
print("Best parameters:", grid_search.best_params_)
print("Best AUC:", grid_search.best_score_)


In [ ]:
class_weight = (1 / y_train.value_counts(normalize=True)).to_dict()

parameters= {'n_estimators': 400, "n_jobs": -1, 'max_depth': 11, 'min_samples_split': 11, 'min_samples_leaf': 6,
             'max_features': 'log2', 'bootstrap': True, "class_weight" : class_weight}

In [ ]:
# 🔹 Train model
model = RandomForestClassifier(**parameters, verbose = 2)
model.fit(X_train, y_train)

In [ ]:
poly_feat_names = ['enc_paym_2', 'enc_loans_credit_type', 'pre_util','pre_loans_credit_cost_rate',
                    'is_zero_loans530', 'pre_loans_credit_limit', 'pre_since_opened', 'is_zero_util', 'pre_pterm']

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df_train.loc[:, important_feat], df_train.flag, stratify=df_train.flag, test_size=0.3)
del df_train
_= gc.collect()

In [ ]:
poly = PolynomialFeatures(degree=2, interaction_only=True, include_bias=False)
poly_feat = poly.fit_transform(X_train[poly_feat_names])
poly_feat

In [ ]:
# Creating a dataframe that contains input features with interactions terms
columns = poly.get_feature_names_out(poly_feat_names)
poly_feat = pd.DataFrame(poly_feat, columns=columns)
poly_feat

In [ ]:
poly_feat.drop(columns=poly_feat_names, axis = 1, inplace=True)
poly_feat

In [ ]:
X_train = pd.concat([X_train.reset_index(), poly_feat], axis=1)

X_train

In [ ]:
X_train = X_train.set_index("index")
X_train

In [ ]:
del poly_feat
gc.collect()

In [ ]:
poly_feat = poly.transform(X_val[poly_feat_names])
poly_feat = pd.DataFrame(poly_feat, columns=columns)
poly_feat.drop(columns=poly_feat_names, axis = 1, inplace=True)
X_val = pd.concat([X_val.reset_index(), poly_feat], axis=1)
X_val = X_val.set_index("index")
X_val

In [ ]:
del poly_feat
gc.collect()

In [ ]:
def objective(trial):
    # Suggest parameters
    param = {
        'objective': 'binary',
        'metric': 'auc',
        'boosting_type': trial.suggest_categorical('boosting_type', ['gbdt', 'goss']),
        'num_leaves': trial.suggest_int('num_leaves', 40, 128),
        'max_depth': trial.suggest_int('max_depth', 6, 11),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-3, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 400, 700),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-8, 1.0),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-8, 1.0),
        'min_child_samples': trial.suggest_int('min_child_samples', 50, 100),
        'random_state': 42,
        'class_weight': 'balanced',
        'n_jobs': -1,
        'verbose': -1   # keep this to silence internal LightGBM logs
    }

    # small sample for optimization
    sample_frac = 0.6
    X_sample, _, y_sample, _ = train_test_split(X_train, y_train, train_size=sample_frac, stratify=y_train, random_state=42)
    X_train_sub, X_val_sub, y_train_sub, y_val_sub = train_test_split(X_sample, y_sample, test_size=0.2, stratify=y_sample, random_state=42)

    # Train
    model = lgb.LGBMClassifier(**param)
    model.fit(
    X_train_sub, y_train_sub,
    eval_set=[(X_val_sub, y_val_sub)],
    eval_metric='auc',
    callbacks=[
        early_stopping(50),     # same as early_stopping_rounds=50
        log_evaluation(100)     # print every 100 iterations
    ]
)

    # Predict
    y_pred = model.predict_proba(X_val_sub)[:, 1]
    score = roc_auc_score(y_val_sub, y_pred)

    return score

In [ ]:
study = optuna.create_study(direction="maximize")  # maximize ROC-AUC
study.optimize(objective, n_trials=20)

print("Best trial:")
print(study.best_trial.params)
print(f"Best ROC-AUC: {study.best_trial.value:.4f}")

In [ ]:
client_stats_train = X_train.groupby('id').agg(
    n_loans=('rn', 'max'),
    avg_util=('pre_util', 'mean')
).reset_index()


# Puis merger
X_train = X_train.merge(client_stats_train, on='id', how='left')

In [ ]:
client_stats_val = X_val.groupby('id').agg(
    n_loans=('rn', 'max'),
    avg_util=('pre_util', 'mean')
).reset_index()


# Puis merger
X_val = X_val.merge(client_stats_val, on='id', how='left')

In [ ]:
X_val, X_val2, y_val, y_val2 = train_test_split(X_val, y_val, stratify=y_val, test_size=0.20)
#del df_train
#_= gc.collect()

In [ ]:
X_val2.shape

In [ ]:
# Définir le modèle XGB avec tes hyperparamètres
def create_xgb_model():
    return XGBClassifier(
        booster='gbtree',
        n_estimators=3500,
        learning_rate=0.05,
        max_depth=6,
        subsample=0.8,
        colsample_bytree=0.9,
        reg_alpha=1,
        reg_lambda=5,
        min_child_weight=40,
        tree_method='gpu_hist',
        predictor='gpu_predictor',
        random_state=42,
        scale_pos_weight=(y_train.value_counts()[0] / y_train.value_counts()[1]),
        eval_metric='auc',
        use_label_encoder=False
    )



In [ ]:
# Paramètres de la cross-validation
n_splits = 5
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

auc_scores = []

for fold, (train_idx, val_idx) in enumerate(cv.split(X_train, y_train), start=1):
    print(f"\n🔹 Fold {fold}/{n_splits}")
    
    X_tr, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    model = create_xgb_model()
    
    start = time()
    model.fit(
        X_tr, y_tr,
        eval_set=[(X_val_fold, y_val_fold)],
        early_stopping_rounds=150,
        verbose=100
    )
    end = time()
    print(f"Training duration for fold {fold}: {end - start:.2f} seconds")
    
    val_preds = model.predict_proba(X_val_fold)[:, 1]
    
    # Calcul AUC sur le fold
    from sklearn.metrics import roc_auc_score
    fold_auc = roc_auc_score(y_val_fold, val_preds)
    print(f"AUC Fold {fold}: {fold_auc:.4f}")
    
    auc_scores.append(fold_auc)

In [ ]:
# Résultat global
print("\n✅ Cross-validation AUC scores:", np.round(auc_scores, 4))
print("✅ Mean AUC:", np.mean(auc_scores))

In [ ]:
auc_scores2 = auc_scores
auc_scores2

In [ ]:
X_train.columns

In [ ]:
# ⚙️ On réduit la taille des ensembles à 50 %
X_train_sub, _, y_train_sub, _ = train_test_split(
    X_train, y_train, train_size=0.2, stratify=y_train, random_state=42
)

X_val_sub, _, y_val_sub, _ = train_test_split(
    X_val, y_val, train_size=0.2, stratify=y_val, random_state=42
)

In [ ]:
def objective(trial):
    # --- Paramètres à optimiser ---
    params = {
        'booster': 'gbtree',
        'n_estimators': 3500,
        'learning_rate': 0.05,
        'tree_method': 'gpu_hist',
        'predictor': 'gpu_predictor',
        'random_state': 42,
        'scale_pos_weight': (y_train.value_counts()[0] / y_train.value_counts()[1]),
        'eval_metric': 'auc',
        'use_label_encoder': False,

        # --- Hyperparamètres optimisés ---
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-4, 10),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-4, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50)
    }

    # --- Entraînement ---
    model = XGBClassifier(**params)
    model.fit(
        X_train_sub, y_train_sub,
        eval_set=[(X_val_sub, y_val_sub)],
        early_stopping_rounds=100,
        verbose=False
    )

    # --- Prédiction et score ---
    y_pred = model.predict_proba(X_val_sub)[:, 1]
    auc = roc_auc_score(y_val_sub, y_pred)

    return auc

In [ ]:
def objective(trial):
    # --- Paramètres de base ---
    params = {
        'objective': 'binary',
        'boosting_type': 'goss',  # gradient boosting classique
        'n_estimators': 3000,
        'learning_rate': 0.05,
        'random_state': 42,
        #'device': 'gpu',  # pour activer GPU
        'metric': 'auc',
        'scale_pos_weight': (y_train.value_counts()[0] / y_train.value_counts()[1]),

        # --- Hyperparamètres à optimiser ---
        'num_leaves': trial.suggest_int('num_leaves', 15, 80),
        'max_depth': trial.suggest_int('max_depth', 4, 10),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'reg_alpha': trial.suggest_loguniform('reg_alpha', 1e-4, 10),
        'reg_lambda': trial.suggest_loguniform('reg_lambda', 1e-4, 10),
        'min_child_samples': trial.suggest_int('min_child_samples', 10, 100)
    }

    # --- Entraînement ---
    model_lgb = lgb.LGBMClassifier(**params)
    model_lgb.fit(
    X_train_sub, y_train_sub,
    eval_set=[(X_val2, y_val2)],
    eval_metric='auc',
    callbacks=[
        lgb.early_stopping(stopping_rounds=100, verbose=False),
        lgb.log_evaluation(100)
    ]
)

    # --- Prédiction et score ---
    y_pred = model_lgb.predict_proba(X_val_sub)[:, 1]
    auc = roc_auc_score(y_val_sub, y_pred)

    return auc


In [ ]:

# --- Lancement de l’optimisation bayésienne ---
start = time()
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30, show_progress_bar=True)
end = time()

print(f"\n✅ Optimisation terminée en {end - start:.1f} secondes")
print("Best AUC:", study.best_value)
print("Best parameters:", study.best_params)


In [ ]:
{'max_depth': 5, 'subsample': 0.995011247757305, 
 'colsample_bytree': 0.623863476660147, 'reg_alpha': 9.80184976286027, 
 'reg_lambda': 0.15121667196893812, 'min_child_weight': 38}

In [ ]:
test_df_or = pd.read_parquet("/kaggle/input/data-tour/test.parquet")

In [ ]:
test_df = test_df_or.copy()
test_df

In [ ]:
paym_cols = [f'enc_paym_{i}' for i in range(10)]

# 1️⃣ Somme des retards
test_df_or['total_delays_25m'] = test_df_or[paym_cols].sum(axis=1)

# 2️⃣ Score pondéré (plus récent = plus important)
weights = np.arange(10, 0, -1)  # [25, 24, ..., 1]
test_df_or['weighted_delay_score'] = test_df_or[paym_cols].mul(weights, axis=1).sum(axis=1)

In [ ]:
test_df_or["time_to_close_gap"] = test_df_or["pre_till_fclose"] - test_df_or["pre_till_pclose"]
test_df_or["loan_duration"] = test_df_or["pre_fterm"] - test_df_or["pre_pterm"]
test_df_or['confirmation_to_open_days'] = test_df_or['pre_since_confirmed'] - test_df_or['pre_since_opened']

In [ ]:
bool_cols = ['is_zero_loans5', 'is_zero_loans530', 'is_zero_loans90']

# Pondérations
weights = [0.2, 0.3, 0.5]

# Calcul du score de risque
test_df_or['risk_score'] = (
    test_df_or['is_zero_loans5'] * weights[0] +
    test_df_or['is_zero_loans530'] * weights[1] +
    test_df_or['is_zero_loans90'] * weights[2]
)
test_df_or["risk_score"]

In [ ]:
submiss = pd.read_parquet("/kaggle/input/submission/sample_submission16.parquet")
submiss

In [ ]:
submiss.rename(columns = {"id": "id_x_rn"}, inplace = True)

In [ ]:
submiss

In [ ]:
X_test_scaled = scaler.transform(test_df_or[cluster_features])
test_df_or['cluster_k13'] = kmeans.predict(X_test_scaled)

In [ ]:
X_test_scaled2 = scaler2.transform(test_df_or[cluster_features_payment])
test_df_or['cluster_payment_k13'] = kmeans2.predict(X_test_scaled2)

In [ ]:
test_df_or = pd.merge(test_df_or, submiss,on = "id_x_rn",
                            how = "inner")
test_df_or

In [ ]:
test_df_labelled = test_df_or[(test_df_or["target"] <= 0.4) | (test_df_or["target"] >= 0.6)]
test_df_labelled

In [ ]:
test_df_labelled["flag"] = test_df_labelled["target"].apply(lambda x: 0 if x <= 0.4 else 1)
test_df_labelled

In [ ]:
test_df_labelled["flag"].value_counts()

In [ ]:
test_df_labelled.drop("target", axis = 1, inplace = True)

In [ ]:
# Séparer X et y dans test_df_labelled2
X_test_labeled = test_df_labelled.drop(columns=['flag'])
y_test_labeled = test_df_labelled['flag']


In [ ]:
# Concaténer avec X_train et y_train
X_train = pd.concat([X_train, test_df_labelled.drop(columns=['flag','id_x_rn'])], axis=0)
y_train = pd.concat([y_train, test_df_labelled['flag']], axis=0)

# Réinitialiser les index
X_train.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)

In [ ]:
X_train.to_parquet("X_train.parquet")
X_val.to_parquet("X_val.parquet")
X_val2.to_parquet("X_val2.parquet")
test_df_or.drop(columns = ["target"]).to_parquet("test_df_ready.parquet")
y_train.to_csv("y_train.csv", index=False)
y_val.to_csv("y_val.csv", index=False)
y_val2.to_csv("y_val2.csv", index=False)

In [ ]:
X_train = pd.read_parquet("/kaggle/input/data-tour-new/X_train.parquet")
X_train

In [ ]:
X_train = pd.read_parquet("/kaggle/input/data-tour-new/X_train.parquet")
X_val   = pd.read_parquet("/kaggle/input/data-tour-new/X_val.parquet")
X_val2  = pd.read_parquet("/kaggle/input/data-tour-new/X_val2.parquet")
test_df_ready = pd.read_parquet("/kaggle/input/data-tour-new/test_df_ready.parquet")

# Chargement des labels (fichiers CSV)
y_train = pd.read_csv("/kaggle/input/data-tour-new/y_train.csv")
y_val   = pd.read_csv("/kaggle/input/data-tour-new/y_val.csv")
y_val2  = pd.read_csv("/kaggle/input/data-tour-new/y_val2.csv")

In [ ]:
X_train = pd.concat([X_train, X_val], axis = 0)
y_train = pd.concat([y_train, y_val], axis = 0)

In [ ]:
#X_train.drop(columns=['id_x_rn'], inplace = True)

In [ ]:
df_train["flag"].value_counts()

In [ ]:
X_train = pd.read_parquet("/kaggle/input/train-new/X_train_new.parquet")
#X_val   = pd.read_parquet("/kaggle/input/data-tour-new/X_val.parquet")
X_val2  = pd.read_parquet("/kaggle/input/data-tour-new/X_val2.parquet")
test_df_ready = pd.read_parquet("/kaggle/input/data-tour-new/test_df_ready.parquet")

# Chargement des labels (fichiers CSV)
#y_train = pd.read_csv("/kaggle/input/train-new/y_train_new.csv")
#y_val   = pd.read_csv("/kaggle/input/data-tour-new/y_val.csv")
y_val2  = pd.read_csv("/kaggle/input/data-tour-new/y_val2.csv")

In [ ]:
X_train = pd.concat([X_train.iloc[:15010858, :], X_train.iloc[15324242:17443422, :]], axis = 0)
X_train.reset_index(drop = True, inplace = True)
X_train.shape

In [ ]:
y_train = pd.concat([pd.read_csv("/kaggle/input/data-tour-new/y_train.csv"), pd.read_csv("/kaggle/input/data-tour-new/y_val.csv")], axis = 0)
y_train.reset_index(drop = True, inplace = True)

In [ ]:
y_train = pd.concat([y_train.iloc[:15010858, :], y_train.iloc[15324242:17443422, :]], axis = 0)
y_train.reset_index(drop = True, inplace = True)
y_train.shape

In [ ]:
def reduce_mem_usage(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object and col_type.name != 'category':
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':
                # entiers positifs
                if c_min >= 0:
                    if c_max < np.iinfo(np.uint8).max:
                        df[col] = df[col].astype(np.uint8)
                    elif c_max < np.iinfo(np.uint16).max:
                        df[col] = df[col].astype(np.uint16)
                    elif c_max < np.iinfo(np.uint32).max:
                        df[col] = df[col].astype(np.uint32)
                    else:
                        df[col] = df[col].astype(np.uint64)
                else:
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
            else:
                # flottants
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print(f"💾 Mémoire réduite de {start_mem:.2f} Mo à {end_mem:.2f} Mo ({100 * (start_mem - end_mem) / start_mem:.1f}% de gain)")
    return df

In [ ]:
X_train = reduce_mem_usage(X_train)
test_df_ready = reduce_mem_usage(test_df_ready)

In [ ]:

start = time()

model_xgb = XGBClassifier(
    booster='gbtree',
    n_estimators=3500,
    learning_rate=0.05,
    max_depth=6,
    subsample=0.99,
    colsample_bytree=0.624,
    reg_alpha=9.8,
    reg_lambda=0.1512,
    min_child_weight=35,
    tree_method='gpu_hist',      # ⚡ utilise le GPU
    predictor='gpu_predictor',   # ⚡ prédiction sur GPU aussi
    random_state=42,
    scale_pos_weight=(y_val.value_counts(normalize=True)[0] / y_val.value_counts(normalize=True)[1]),
    eval_metric='auc'
)

model_xgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    early_stopping_rounds=150,
    verbose=100
)

end = time()
print(f"Training duration: {end - start:.2f} seconds")

In [ ]:
val_preds_proba = model_xgb.predict_proba(X_val)[:, 1]   # probability of positive class
train_preds_proba = model_xgb.predict_proba(X_train)[:, 1]

In [ ]:
train_auc = roc_auc_score(y_train, train_preds_proba)
val_auc = roc_auc_score(y_val, val_preds_proba)
print(f"Train AUC: {train_auc}\n Val AUC: {val_auc}")

In [ ]:
X_val2.reset_index(drop = True, inplace = True)
y_val2.reset_index(drop = True, inplace = True)

In [ ]:
val_preds_proba2 = model_xgb.predict_proba(X_val2)[:, 1]
val_auc2 = roc_auc_score(y_val2, val_preds_proba2)
val_auc2

In [ ]:
# Explainer basé sur le modèle et le dataset
explainer = shap.Explainer(model_xgb, X_val2)
shap_values = explainer(X_val2)

In [ ]:
shap.summary_plot(shap_values[1], X_val2)

In [ ]:
roc_auc_score(y_train, model_xgb.predict_proba(X_train)[:, 1])

In [ ]:
val_preds_proba2_df = pd.DataFrame({"proba": val_preds_proba2})
val_preds_proba2_df

In [ ]:
X_val2_new = pd.concat([X_val2, val_preds_proba2_df], axis = 1)
X_val2_new

In [ ]:
val2_df = pd.concat([X_val2_new["proba"], y_val2 ], axis = 1)
val2_df

In [ ]:
val2_df["flag"].value_counts()

In [ ]:
val2_df[(val2_df["proba"]>0.7) & (val2_df["flag"] == 0)]

In [ ]:
# Importance des features
importance = model_xgb.feature_importances_

# Si tu veux les associer aux noms de colonnes
feature_importance_df = pd.DataFrame({
    "feature": X_train.columns,
    "importance": importance
}).sort_values(by="importance", ascending=False)

feature_importance_df.head(40)

In [ ]:
len(X_val.columns)

In [ ]:
X_val.shape

{'****num_leaves': 40, 'max_depth': 9, 'subsample': 0.8408296425836999, 'colsample_bytree': 0.7743312411714467, 'reg_alpha': 0.000846628303050795, 'reg_lambda': 0.4931808762375825, 'min_child_samples': 25}****

In [ ]:
y_train = y_train.values.ravel()
y_val2 = y_val2.values.ravel()

In [ ]:
y_train = pd.DataFrame(y_train, columns=["flag"])
y_val2 = pd.DataFrame(y_val2, columns=["flag"])

In [5]:
y_train = y_train.squeeze()
y_val2 = y_val2.squeeze()

NameError: name 'y_val2' is not defined

In [8]:
import lightgbm as lgb
print(lgb.__version__)

# Test du support GPU
print("GPU support:", lgb.basic._ConfigAliases().get("device_type"))


4.6.0
GPU support: {'device_type', 'device'}


In [10]:
start = time()
model_lgbm = lgb.LGBMClassifier(boosting_type= 'gbdt', num_leaves= 128, max_depth= 9, 
                                learning_rate= 0.05, n_estimators= 3500,
    subsample=0.84,
    colsample_bytree=0.77,
    reg_alpha=0.00084,
    reg_lambda=5,
    device_type='gpu',
    gpu_platform_id = 0,
    gpu_device_id = 0,
    min_child_samples=25,
    class_weight=(1 / y_val.value_counts(normalize=True)).to_dict(),
    random_state=42, verbose = -1)

model_lgbm.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='auc',
    callbacks=[
        early_stopping(stopping_rounds=150),
        log_evaluation(period=100)
    ]
)
end = time()

print(f"Duration: {end - start}")

Training until validation scores don't improve for 150 rounds
[100]	valid_0's auc: 0.65508	valid_0's binary_logloss: 0.65247


KeyboardInterrupt: 

In [ ]:
start = time()
model_lgbm = lgb.LGBMClassifier(boosting_type= 'goss', num_leaves= 128, max_depth= 7, 
                                learning_rate= 0.05, n_estimators= 3500,
    subsample=0.9,
    colsample_bytree=0.9,
    reg_alpha=0.3,
    reg_lambda=2,
    min_child_samples=30,
    class_weight=(1 / y_val2.value_counts(normalize=True)).to_dict(),
    random_state=42, verbose = -1)

model_lgbm.fit(
    X_train, y_train,
    eval_set=[(X_val2, y_val2)],
    eval_metric='auc',
    callbacks=[
        early_stopping(stopping_rounds=150),
        log_evaluation(period=100)
    ]
)
end = time()

print(f"Duration: {end - start}")

In [ ]:
y_train.value_counts()

In [ ]:
val_preds_proba = model_lgbm.predict_proba(X_val, num_iteration=model_lgbm.best_iteration_)[:, 1]
train_preds_proba = model_lgbm.predict_proba(X_train, num_iteration=model_lgbm.best_iteration_)[:, 1]

In [ ]:
train_auc = roc_auc_score(y_train, train_preds_proba)
val_auc = roc_auc_score(y_val, val_preds_proba)
print(f"Train AUC: {train_auc}\n Val AUC: {val_auc}")

In [ ]:
train_auc = roc_auc_score(y_train, train_preds_proba)
val_auc = roc_auc_score(y_val, val_preds_proba)
print(f"Train AUC: {train_auc}\n Val AUC: {val_auc}")

In [ ]:
train_auc = roc_auc_score(y_train, train_preds_proba)
val_auc2 = roc_auc_score(y_val2, val_preds_proba2)
print(f"Train AUC: {train_auc}\n Val AUC: {val_auc2}")

In [ ]:
X_train.columns

In [ ]:
test_df_or = pd.read_parquet("../input/test.parquet")
test_df_or

In [ ]:
test_df = test_df_or.copy()

In [ ]:
paym_cols = [f'enc_paym_{i}' for i in range(10)]

# 1️⃣ Somme des retards
test_df['total_delays_25m'] = test_df[paym_cols].sum(axis=1)

# 2️⃣ Score pondéré (plus récent = plus important)
weights = np.arange(10, 0, -1)  # [25, 24, ..., 1]
test_df['weighted_delay_score'] = test_df[paym_cols].mul(weights, axis=1).sum(axis=1)

In [ ]:
X_train.columns

In [ ]:
test_df["time_to_close_gap"] = test_df["pre_till_fclose"] - test_df["pre_till_pclose"]
test_df["loan_duration"] = test_df["pre_fterm"] - test_df["pre_pterm"]
test_df['confirmation_to_open_days'] = test_df['pre_since_confirmed'] - test_df['pre_since_opened']

In [ ]:
bool_cols = ['is_zero_loans5', 'is_zero_loans530', 'is_zero_loans90']

# Pondérations
weights = [0.2, 0.3, 0.5]

# Calcul du score de risque
test_df['risk_score'] = (
    test_df['is_zero_loans5'] * weights[0] +
    test_df['is_zero_loans530'] * weights[1] +
    test_df['is_zero_loans90'] * weights[2]
)
test_df["risk_score"]

In [ ]:
test_df.drop(["enc_paym_17", "enc_paym_15","pre_loans5", "enc_paym_18", 
               "is_zero_maxover2limit", "is_zero_over2limit", "is_zero_loans6090",
               "enc_loans_account_cur", "pre_loans90", "pre_loans3060", 
               "pre_loans6090", "pre_loans_total_overdue", "enc_paym_22", "enc_paym_21"], axis = 1, inplace = True)

In [ ]:
test_df["limit_x_util"] = test_df["pre_loans_credit_limit"] * test_df["pre_util"]
test_df["cost_x_util"] = test_df["pre_loans_credit_cost_rate"] * test_df["pre_util"]
test_df["gap_x_duration"] = test_df["time_to_close_gap"] * test_df["loan_duration"]

In [ ]:
X_test_scaled = scaler.transform(test_df[cluster_features])
test_df['cluster_k13'] = kmeans.predict(X_test_scaled)

In [ ]:
X_test_scaled2 = scaler2.transform(test_df[cluster_features_payment])
test_df['cluster_payment_k13'] = kmeans2.predict(X_test_scaled2)

In [ ]:
test_preds_proba = model_lgbm.predict_proba(test_df_ready [X_train.columns])[:, 1]

In [ ]:
test_df_or.drop(columns = ["target"], inplace = True)

In [ ]:
test_preds_proba = model_xgb.predict_proba(test_df_or[X_train.columns])[:, 1]

In [ ]:
len(test_df.drop(["id_x_rn"], axis = 1).columns)

In [ ]:
len(X_train.columns)

In [ ]:
test_preds_proba_df = pd.DataFrame({"target": test_preds_proba})
submission_df = pd.concat([test_df_ready ["id_x_rn"], test_preds_proba_df], axis = 1)
submission_df.rename(columns={"id_x_rn": "id"}, inplace=True)
submission_df

In [ ]:
submission_df[(submission_df["target"] <= 0.4) | (submission_df["target"] >= 0.6)]

In [ ]:
submission_df5 = submission_df.copy()

In [ ]:
submission_df4 = submission_df.copy()

In [ ]:
submission_df3 = submission_df.copy()

In [ ]:
submission_df2 = submission_df.copy()

In [ ]:
submission_df2

In [ ]:
submission_df2[(submission_df2["target"] <= 0.45) | (submission_df2["target"] >= 0.55)]

In [ ]:
X_train.iloc[:,45:55].hist()

In [ ]:
submission_df.to_parquet("sample_submission.parquet")

In [ ]:
y_val.value_counts()

In [ ]:
import xgboost
print(xgboost.__version__)

In [ ]:
from sklearn.utils import resample
import pandas as pd

# 1️⃣ Combine X_val et y_val pour faciliter le traitement
val_data = X_val.copy()
val_data['target'] = y_val.values

# 2️⃣ Séparer les deux classes
val_0 = val_data[val_data['target'] == 0]
val_1 = val_data[val_data['target'] == 1]

# 3️⃣ Trouver la taille minimale (pour équilibrer)
min_size = min(len(val_0), len(val_1))

# 4️⃣ Échantillonner aléatoirement la même quantité dans chaque classe
val_0_bal = resample(val_0, n_samples=min_size, random_state=42, replace=False)
val_1_bal = resample(val_1, n_samples=min_size, random_state=42, replace=False)

# 5️⃣ Fusionner et mélanger
val_balanced = pd.concat([val_0_bal, val_1_bal]).sample(frac=1, random_state=42)

# 6️⃣ Re-séparer en X et y
X_val_bal = val_balanced.drop('target', axis=1)
y_val_bal = val_balanced['target']

print("Shape balanced:", X_val_bal.shape)
print("Class distribution:\n", y_val_bal.value_counts(normalize=True))


In [ ]:
submission_df[(submission_df["target"] >= 0.55) | (submission_df["target"] <= 0.45)]

In [ ]:
submission_df.to_parquet("sample_submission.parquet", index=False)

In [ ]:
import xgboost as xgb

# Modèle XGBoost
model_xgb = xgb.XGBClassifier(
    booster='gbtree',
    objective='binary:logistic',
    eval_metric='auc',
    learning_rate=0.07,
    max_depth=7,
    n_estimators=1800,
    subsample=0.7571512794863664,
    colsample_bytree=0.6952547009268857,
    reg_alpha=0.3,
    reg_lambda=0.3,
    min_child_weight=30,  # équivalent LightGBM
    scale_pos_weight=y_val.value_counts(normalize=True)[0] / y_val.value_counts(normalize=True)[1],
    random_state=42,
    n_jobs=-1
)

In [ ]:


# Entraînement avec early stopping intégré
model_xgb.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=100,                 # affiche tous les 100 rounds
    early_stopping_rounds=50     # stoppe si pas d’amélioration sur 50 rounds
)


In [18]:
# Importance des features
importance = model_lgbm.feature_importances_

# Si tu veux les associer aux noms de colonnes
feature_importance_df = pd.DataFrame({
    "feature": X_train.columns,
    "importance": importance
}).sort_values(by="importance", ascending=False)

feature_importance_df.head(40)

,feature,importance
0,id,21224
1,rn,12582
13,pre_loans_credit_cost_rate,9589
8,pre_loans_credit_limit,9157
2,pre_since_opened,8580
3,pre_since_confirmed,8101
4,pre_pterm,8087
6,pre_till_pclose,7668
5,pre_fterm,6796
7,pre_till_fclose,6512


In [19]:
import joblib

In [20]:
joblib.dump(model_lgbm, "model_lgbm.pickle")

['model_lgbm.pickle']

In [ ]:
# Importance des features
importance = model_lgbm.feature_importances_

# Si tu veux les associer aux noms de colonnes
feature_importance_df = pd.DataFrame({
    "feature": X_train.columns,
    "importance": importance
}).sort_values(by="importance", ascending=False)

print(feature_importance_df)

In [ ]:
# Importance des features
importance = model_xgb.feature_importances_

# Si tu veux les associer aux noms de colonnes
feature_importance_df = pd.DataFrame({
    "feature": X_train.columns,
    "importance": importance
}).sort_values(by="importance", ascending=False)

print(feature_importance_df)

In [ ]:
feature_importance_df.tail(20)

In [ ]:
feature_importance_df.head(14)

In [ ]:
["enc_paym_17", "enc_paym_15","pre_loans5", "enc_paym_18", 
               "is_zero_maxover2limit", "is_zero_over2limit", "is_zero_loans6090",
               "enc_loans_account_cur", "pre_loans90", "pre_loans3060", 
               "pre_loans6090", "pre_loans_total_overdue", "enc_paym_22", "enc_paym_21"]

In [ ]:
# Importance des features
importance = model_lgbm.feature_importances_

# Si tu veux les associer aux noms de colonnes
feature_importance_df = pd.DataFrame({
    "feature": X_train.columns,
    "importance": importance
}).sort_values(by="importance", ascending=False)

print(feature_importance_df)

In [ ]:
feature_importance_df.tail(20)

In [ ]:
feature_importance_df.tail(20)

In [ ]:
# supposons que df_importance contient 'feature' et 'importance'
feature_importance_df['cum_importance'] = feature_importance_df['importance'].cumsum() / feature_importance_df['importance'].sum()

# garder features qui couvrent 90% de l'importance
selected_features = feature_importance_df[feature_importance_df['cum_importance'] <= 0.9]['feature'].tolist()
selected_features